# 네이버 영화평 감성분석 _ 정확도 평가

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

1. 데이터 가져오기

2. 데이터 전처리
    - Null 데이터 확인 및 제거
    - 중복 데이터 확인 및 제거

3. 텍스트 전처리
    - 한글 이외 문자 공백 처리하고 strip
    - '' 와 같은 값 Null 값으로 변경 후 제거 (np.nan)
    - 전처리 완료 후 데이터 파일로 저장

In [2]:
# 전처리 완료해서 저장한 데이터 불러오기
train_df = pd.read_csv('data/naver_movie_train_전처리완료.csv', sep='\t')
test_df = pd.read_csv('data/naver_movie_test_전처리완료.csv', sep='\t')

4. 한글 처리 

In [3]:
import joblib
scores = joblib.load('data/scores.pkl') # Soynlp 로 토큰화 -> 학습 결과 불러오기 

In [4]:
from soynlp.tokenizer import MaxScoreTokenizer
max_tokenizer = MaxScoreTokenizer(scores) # 학습한 데이터 넣어서 토큰화 객체 생성

In [5]:
# 불용어(stopwords) 리스트 만들기
# 출처: https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART002390885

with open('data/한국어조사.txt', encoding='utf-8') as file:
    stopwords = file.read()

stop_list = stopwords.split(', ')


In [6]:
# 아무 텍스트 가져와서 한글 처리 과정 해보기
text = '혼란의 시대. 과학을 부정하는 정치병자들에게 날리는 불꽃혜성 인류 종말쑈! 인류의 생존까지도 경제적 정치적 도박의 소재로 삼는 탐욕스런 자본과 정치병자들이 결탁하여 감상적인 대중을 선동할때, 이미 이 지구에서 이성적 인류는 멸종위기종. 결국 자본의 탐욕과 결합한 정치병자들을 치유하기 위해서라도 우리는 합리적 과학에 근거한 이성의 정치를 끝까지 포기하지 말고 사수해야 한다.'
word_list = max_tokenizer.tokenize(text)
word_list = [word for word in word_list if word not in stop_list]
review = ' '.join(word_list)
review

'혼란 시대. 과학 부정 하는 정치 병자 들에게 날리는 불꽃혜성 인류 종말쑈! 인류의 생존 경제 정치적 도박의 소재로 삼는 탐욕스런 자본과 정치 병자들이 결탁하여 감상적인 대중을 선동할때, 이미 지구 이성적 인류는 멸종위기 종. 결국 자본의 탐욕과 결합한 정치 병자들을 치유하기 위해 서라도 합리적 과학 근거 이성의 정치 끝 포기하지 사수 해야 한다 .'

5. X_train, X_test, y_train, y_test 만들기

In [7]:
# X_train
from tqdm.notebook import tqdm
X_train = []
for document in tqdm(train_df.document):
    word_list = max_tokenizer.tokenize(document) # 토큰화
    word_list = [word for word in word_list if word not in stop_list] # 불용어 제거
    review = ' '.join(word_list)
    X_train.append(review)

  0%|          | 0/145393 [00:00<?, ?it/s]

In [8]:
# X_test
from tqdm.notebook import tqdm
X_test = []
for document in tqdm(test_df.document):
    word_list = max_tokenizer.tokenize(document) # 토큰화
    word_list = [word for word in word_list if word not in stop_list] # 불용어 제거
    review = ' '.join(word_list)
    X_test.append(review)

  0%|          | 0/48852 [00:00<?, ?it/s]

In [9]:
# y_train, y_test 
y_train = train_df.label.values
y_test = test_df.label.values

6. Feature 변환 - CountVectorizer

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

cvect = CountVectorizer()
cvect.fit(X_train)
X_trained = cvect.transform(X_train)
X_tested = cvect.transform(X_test)

In [11]:
X_trained.shape, X_tested.shape

((145393, 159347), (48852, 159347))

7. 모델 생성 / 학습 / 평가 --- 분류기 선택

- Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=2022)
lrc.fit(X_trained, y_train)
lrc.score(X_tested, y_test)

0.8291369851797266

- SVM

In [13]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_trained, y_train)
svc.score(X_tested, y_test)

0.8244493572422829

- KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_trained, y_train)
knn.score(X_tested, y_test)

0.6931138950298862

- Random Forest

In [15]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2021)
rfc.fit(X_trained, y_train)
rfc.score(X_tested, y_test)

0.7974494391222468

- Ensemble

In [31]:
from sklearn.ensemble import VotingClassifier

voc = VotingClassifier(
    estimators=[('LRC', lrc),('SVC', svc), ('KNN', knn)], voting='hard'
)

In [32]:
voc.fit(X_trained, y_train)
voc.score(X_tested, y_test)

0.8266396462785557

- Naive Bayes 

In [18]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [19]:
cvect2 = CountVectorizer(min_df=1, max_df=0.9, ngram_range=(1,2))
cvect2.fit(X_train)
X_trained2 = cvect2.transform(X_train)
X_tested2 = cvect2.transform(X_test)
X_trained2.shape, X_tested2.shape

((145393, 900051), (48852, 900051))

In [20]:
nb.fit(X_trained2, y_train)
nb.score(X_tested2, y_test)

0.846045197740113

8. 하이퍼 파라미터 튜닝

- Logistic Regression

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [22]:
cvect.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [27]:
pipeline = Pipeline([
    ('cvect', CountVectorizer()),
    ('lrc', LogisticRegression(random_state=2022))
])
params = {
    'cvect__ngram_range' : [(1,1), (1,2)],
    'cvect__max_df' : [0.8, 0.85, 0.9],  # 소수점 == % / 정수 == 단어수
    'cvect__min_df' : [0, 1, 2],
    'lrc__C' : [0.1, 0.5, 1]
}

In [28]:
grid_pipe = GridSearchCV(
    pipeline, param_grid=params, scoring='accuracy', cv=3
)
%time grid_pipe.fit(X_train, y_train)

Wall time: 25min 2s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('cvect', CountVectorizer()),
                                       ('lrc',
                                        LogisticRegression(random_state=2022))]),
             param_grid={'cvect__max_df': [0.8, 0.85, 0.9],
                         'cvect__min_df': [0, 1, 2],
                         'cvect__ngram_range': [(1, 1), (1, 2)],
                         'lrc__C': [0.1, 0.5, 1]},
             scoring='accuracy')

In [29]:
grid_pipe.best_params_

{'cvect__max_df': 0.8,
 'cvect__min_df': 0,
 'cvect__ngram_range': (1, 2),
 'lrc__C': 1}

In [30]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.8383894211086547

- SVM

In [34]:
svc.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [ ]:
pipeline = Pipeline([
    ('cvect', CountVectorizer()),
    ('lrc', LogisticRegression(random_state=2022))
])
params = {
    'cvect__ngram_range' : [(1,1), (1,2)],
    'cvect__max_df' : [0.9, 0.99],  # 소수점 == % / 정수 == 단어수
    'cvect__min_df' : [1, 5],
    'svc__C' : [0.1, 0.5, 1]
}

- KNN

In [35]:
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [40]:
pipeline = Pipeline([
    ('cvect', CountVectorizer()),
    ('knn', KNeighborsClassifier())
])
params = {
    'cvect__ngram_range' : [(1,1), (1,2)],
    'cvect__max_df' : [0.9, 0.99],  # 소수점 == % / 정수 == 단어수
    'cvect__min_df' : [1, 5],
    'knn__n_neighbors' : [5, 7, 11],
    'knn__p' : [1, 2]
}

In [41]:
grid_pipe = GridSearchCV(
    pipeline, param_grid=params, scoring='accuracy', cv=3
)
%time grid_pipe.fit(X_train, y_train)

Wall time: 5h 37min 26s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('cvect', CountVectorizer()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'cvect__max_df': [0.9, 0.99], 'cvect__min_df': [1, 5],
                         'cvect__ngram_range': [(1, 1), (1, 2)],
                         'knn__n_neighbors': [5, 7, 11], 'knn__p': [1, 2]},
             scoring='accuracy')

In [42]:
grid_pipe.best_params_

{'cvect__max_df': 0.9,
 'cvect__min_df': 5,
 'cvect__ngram_range': (1, 2),
 'knn__n_neighbors': 5,
 'knn__p': 1}

In [43]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.7023868009498075